Perusahaan peminjaman sepeda di San Fransisco memintamu untuk menganalisa durasi perjalanan customer mereka. Untuk memudahkan analisa, lakukan hal-hal berikut ini:

> Add blockquote



## **STATISTIK DESKRIPTIF**

1. Lakukan perhitungan central tendency (mean, median, dan modus).
2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).
3. Lakukan pengecekan skewness dan kurtosis.
3. Analisa apakah ada data outlier berdasarkan sebaran datanya.
4. Cari tahu berapa persentase outlier terhadap data keseluruhan.
5. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

## **API**

Buatlah API untuk:
- `Menampilkan` seluruh entry data setelah dilakukan handling outlier.
- `Menghapus` entry data, dari data yang telah dihandling outlier.

clue:
- Load data csv yang sudah diolah dengan pandas, kemudian konversi data ke dictionary `df.to_dict()` atau json `df.to_json()` untuk dapat diolah lebih lanjut dengan API menggunakan FastAPI.
- Pembuatan API dilakukan pada script `.py`
- Endpoint dan penamaan function pada API dibebaskan.


---



Data dapat diakses dari BigQuery dengan ketentuan:

- Project_id: `bigquery-public-data`
- Dataset: `san_francisco_bikeshare`
- Table: `bikeshare_trips`
- Ambil hanya kolom `duration_sec` saja
- Berikan LIMIT pada data yang diambil sebesar 3000 entry data

Koneksikan data di atas menggunakan `BigQuery` ke Google Colab dengan code berikut:
```py
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "rock-wonder-317907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
```

Untuk melakukan Query menggunakan cara ini, kamu dapat menggunakan method `client.query('Masukkan Querynya').to_dataframe()`. Outputnya akan berupa Pandas dataframe, sehingga harus import Pandas.

Contoh:

```py
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at < "2022-07-01"
ORDER BY year,month ASC
''').to_dataframe()
```



# ANSWER STATISTIK DESKRIPTIF

## Preparation

In [42]:
# Working area
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "hacktiv8-022" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


In [43]:
df = client.query('''
SELECT duration_sec
FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3000;
''').to_dataframe()

df

,duration_sec
0,788
1,560
2,965
3,497
4,489
...,...
2995,148
2996,399
2997,280
2998,338


In [44]:
import pandas as pd
from scipy import stats
from scipy.stats import skew, kurtosis


In [45]:
df = pd.DataFrame(df)

## 1. Lakukan perhitungan central tendency (mean, median, dan modus).

In [46]:
# Menghitung mean
mean_value = df['duration_sec'].mean()

# Menghitung median
median_value = df['duration_sec'].median()

# Menghitung modus
mode_value = stats.mode(df['duration_sec'])[0]

In [47]:
print(f"Mean: {mean_value}")
print(f"Median: {median_value}")
print(f"Modus: {mode_value}")

Mean: 816.415
Median: 501.0
Modus: 325


## 2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).

In [48]:
# Menghitung standar deviasi
std_dev = df['duration_sec'].std()

# Menghitung variance
variance = df['duration_sec'].var()

# Menghitung range
range_value = df['duration_sec'].max() - df['duration_sec'].min()

In [49]:
print(f"Standard Deviation: {std_dev}")
print(f"Variance: {variance}")
print(f"Range: {range_value}")

Standard Deviation: 2873.99854940737
Variance: 8259867.661995665
Range: 67486


## 3. Lakukan pengecekan skewness dan kurtosis.

In [50]:
# Menghitung skewness
skewness_value = skew(df['duration_sec'])

# Menghitung kurtosis
kurtosis_value = kurtosis(df['duration_sec'])

In [51]:
print(f"Skewness: {skewness_value}")
print(f"Kurtosis: {kurtosis_value}")

Skewness: 18.81860329770836
Kurtosis: 389.51479869298896


## 4. Analisa apakah ada data outlier berdasarkan sebaran datanya.

- Berdasarkan perhitungan central tendency, bisa dilihat value `mean` lebih besar dari pada `median` dan `modus`, yang indikasinya ke `skewness positif` karena beberapa value yang sangat tinggi (outlier) menarik rata-rata ke arah kanan.
- Berdasarkan pengecekan dispresi, `range` yang besar menunjukan ada rentang value yang luas pada dataset. Dan standar deviasi yang didapatkan jauh tinggi dibanding rata-rata, yang menandakan data sangat besar dan beberapa value sangat jauh dari pusat distribusi.
- Berdasarkan value kurtosis yang tinggi, tandanya distribusinya `leptokurtik`, yang artinya ada outlier extrim di kedua ekor distribusi, khususnya di sisi atas karena nilai durasi yang jauh lebih besar.

## 5. Cari tahu berapa persentase outlier terhadap data keseluruhan.

In [52]:
# Menggunakan metode Z-score (berdasarkan standar deviasi)
upper_bound = mean_value + 3 * std_dev
lower_bound = 0  # Karena durasi tidak bisa negatif

# Menghitung jumlah outliers berdasarkan Z-score
outliers_z = df[(df['duration_sec'] > upper_bound)]
percent_outliers_z = (len(outliers_z) / len(df)) * 100

print(f"Persentase outliers (Z-score): {percent_outliers_z}%")

Persentase outliers (Z-score): 0.5%


Hanya sekitar 0.5% dari data yang dianggap sebagai outliers, yang menunjukkan kalau mayoritas data berada dalam 3 standar deviasi dari mean, tetapi ada beberapa value yang jauh dari mean.

In [53]:
# Menggunakan metode IQR
Q1 = df['duration_sec'].quantile(0.25)
Q3 = df['duration_sec'].quantile(0.75)
IQR = Q3 - Q1
lower_bound_iqr = Q1 - 1.5 * IQR
upper_bound_iqr = Q3 + 1.5 * IQR

# Menghitung jumlah outliers berdasarkan IQR
outliers_iqr = df[(df['duration_sec'] < lower_bound_iqr) | (df['duration_sec'] > upper_bound_iqr)]
percent_outliers_iqr = (len(outliers_iqr) / len(df)) * 100

print(f"Persentase outliers (IQR): {percent_outliers_iqr}%")

Persentase outliers (IQR): 5.566666666666667%


Sekitar 5.57% data yang dianggap outliers, persentase outliers yang lebih tinggi dibandingkan dengan Z-score menunjukkan bahwa metode ini mengambil sample lebih banyak data yang jauh dari rentang normal.

## 6. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

In [54]:
# Menghapus data outliers
df_no_outliers = df[(df['duration_sec'] >= lower_bound_iqr) & (df['duration_sec'] <= upper_bound_iqr)]

# Statistik pemusatan setelah penghapusan outlier
mean_after = df_no_outliers['duration_sec'].mean()
median_after = df_no_outliers['duration_sec'].median()
mode_after = stats.mode(df_no_outliers['duration_sec'])[0]

# 4. Perbandingan sebelum dan sesudah
print(f"Perubahan Mean: {mean_value} -> {mean_after}")
print(f"Perubahan Median: {median_value} -> {median_after}")
print(f"Perubahan Modus: {mode_value} -> {mode_after}")

Perubahan Mean: 816.415 -> 549.3353335686552
Perubahan Median: 501.0 -> 481.0
Perubahan Modus: 325 -> 325


Perbedaan antara `mean` dan `median` mengecil setelah penghapusan outlier, yang menunjukkan sekarang distribusinya lebih seimbang.


# ANSWER API

In [55]:
df_no_outliers.to_csv('P0_Kisi2_LC3_data_after_outlier_handling.csv', index=False)